In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Квантовий алгоритм наближеної оптимізації

*Оцінка використання: 22 хвилини на процесорі Heron r3 (ПРИМІТКА: Це лише оцінка. Ваш час виконання може відрізнятися.)*
## Передумови
Цей посібник демонструє, як реалізувати **Квантовий алгоритм наближеної оптимізації (QAOA)** – гібридний (квантово-класичний) ітеративний метод – у контексті патернів Qiskit. Ви спочатку розв'яжете задачу **Максимального розрізу** (або **Max-Cut**) для невеликого графа, а потім навчитеся виконувати її на утилітарному масштабі. Всі виконання на обладнанні у цьому посібнику мають виконуватися в межах часового ліміту для безкоштовного Відкритого плану.

Задача Max-Cut є задачею оптимізації, яку важко розв'язати (точніше, це *NP-важка* задача) з низкою різних застосувань у кластеризації, мережевій науці та статистичній фізиці. Цей посібник розглядає граф вузлів, з'єднаних ребрами, і має на меті розбити вузли на два набори таким чином, щоб кількість ребер, які перетинає цей розріз, була максимізована.

![Illustration of a max-cut problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Вимоги
Перед початком цього посібника переконайтеся, що у Вас встановлено наступне:
- Qiskit SDK v1.0 або пізніше, з підтримкою [візуалізації](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 або пізніше (`pip install qiskit-ibm-runtime`)

Крім того, Вам знадобиться доступ до екземпляра на [IBM Quantum Platform](/guides/cloud-setup). Зверніть увагу, що цей посібник не можна виконати на Відкритому плані, оскільки він запускає робочі навантаження з використанням [сесій](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session), які доступні лише з доступом до Преміум плану.
## Налаштування

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Частина I. Малошкалюючий QAOA
Перша частина цього посібника використовує малошкалюючу задачу Max-Cut для ілюстрації кроків для розв'язання задачі оптимізації з використанням квантового комп'ютера.

Щоб надати певний контекст перед відображенням цієї задачі на квантовий алгоритм, Ви можете краще зрозуміти, як задача Max-Cut стає класичною комбінаторною задачею оптимізації, спочатку розглянувши мінімізацію функції $f(x)$

$$
\min_{x\in {0, 1}^n}f(x),
$$

де вхід $x$ є вектором, компоненти якого відповідають кожному вузлу графа. Потім обмежте кожний з цих компонентів до $0$ або $1$ (які представляють включення або невключення до розрізу). Цей малошкалюючий приклад використовує граф з $n=5$ вузлами.

Ви можете написати функцію пари вузлів $i,j$, яка вказує, чи відповідне ребро $(i,j)$ знаходиться в розрізі. Наприклад, функція $x_i + x_j - 2 x_i x_j$ дорівнює 1 тільки якщо один з $x_i$ або $x_j$ дорівнює 1 (що означає, що ребро знаходиться в розрізі) і нулю в іншому випадку. Задачу максимізації ребер у розрізі можна сформулювати як

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

що можна переписати як мінімізацію у вигляді

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

Мінімум $f(x)$ у цьому випадку досягається, коли кількість ребер, що перетинаються розрізом, максимальна. Як Ви бачите, поки що немає нічого, що стосується квантових обчислень. Вам потрібно переформулювати цю задачу у щось, що квантовий комп'ютер може зрозуміти.
Ініціалізуйте Вашу задачу, створивши граф з $n=5$ вузлами.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Крок 1: Відображення класичних входів у квантову задачу
Перший крок патерну – це відображення класичної задачі (графа) у квантові **контури** та **оператори**. Для цього необхідно виконати три основні кроки:

1. Використати серію математичних переформулювань для представлення цієї задачі з використанням нотації задач Квадратичної необмеженої бінарної оптимізації (QUBO).
2. Переписати задачу оптимізації як гамільтоніан, для якого основний стан відповідає розв'язку, що мінімізує функцію вартості.
3. Створити квантовий контур, який підготує основний стан цього гамільтоніана через процес, подібний до квантового відпалу.

**Примітка:** У методології QAOA Ви зрештою хочете мати оператор (**гамільтоніан**), який представляє **функцію вартості** нашого гібридного алгоритму, а також параметризований контур (**Ansatz**), який представляє квантові стани з кандидатами на розв'язки задачі. Ви можете отримати вибірку з цих стан-кандидатів, а потім оцінити їх, використовуючи функцію вартості.

#### Граф &rarr; задача оптимізації
Перший крок відображення – це зміна нотації. Наступне виражає задачу в нотації QUBO:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

де $Q$ є матрицею $n\times n$ дійсних чисел, $n$ відповідає кількості вузлів у Вашому графі, $x$ є вектором бінарних змінних, введеним вище, і $x^T$ вказує на транспонування вектора $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Задача оптимізації &rarr; Гамільтоніан
Потім Ви можете переформулювати задачу QUBO як **гамільтоніан** (тут, матрицю, яка представляє енергію системи):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Кроки переформулювання від задачі QAOA до гамільтоніана**
</summary>

Щоб продемонструвати, як задачу QAOA можна переписати в цій формі, спочатку замініть бінарні змінні $x_i$ на новий набір змінних $z_i\in{-1, 1}$ через

$$
x_i = \frac{1-z_i}{2}.
$$

Тут Ви можете побачити, що якщо $x_i$ дорівнює $0$, то $z_i$ має бути $1$. Коли $x_i$ замінюються на $z_i$ в задачі оптимізації ($x^TQx$), можна отримати еквівалентне формулювання.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Тепер, якщо ми визначимо $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$, видалимо префактор і константний член $n^2$, ми прийдемо до двох еквівалентних формулювань однієї задачі оптимізації.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Тут $b$ залежить від $Q$. Зауважте, що для отримання $z^TQz + b^Tz$ ми відкинули множник 1/4 і константне зміщення $n^2$, які не відіграють ролі в оптимізації.

Тепер, щоб отримати квантове формулювання задачі, підвищимо змінні $z_i$ до матриці Паулі $Z$, такої як матриця $2\times 2$ вигляду

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Коли Ви підставляєте ці матриці в задачу оптимізації вище, Ви отримуєте наступний гамільтоніан

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Також пам'ятайте, що матриці $Z$ вбудовані в обчислювальний простір квантового комп'ютера, тобто, простір Гільберта розміру $2^n\times 2^n$. Таким чином, Ви повинні розуміти такі терми як $Z_iZ_j$ як тензорний добуток $Z_i\otimes Z_j$, вбудований у простір Гільберта $2^n\times 2^n$. Наприклад, у задачі з п'ятьма змінними рішення терм $Z_1Z_3$ розуміється як $I\otimes Z_3\otimes I\otimes Z_1\otimes I$, де $I$ є одиничною матрицею $2\times 2$.*
</details>

Цей гамільтоніан називається **гамільтоніаном функції вартості**. Він має властивість, що його основний стан відповідає розв'язку, який **мінімізує функцію вартості $f(x)$**.
Тому, щоб розв'язати Вашу задачу оптимізації, Вам тепер потрібно підготувати основний стан $H_C$ (або стан з високим перекриттям з ним) на квантовому комп'ютері. Потім отримання вибірки з цього стану з високою ймовірністю дасть розв'язок до $min~f(x)$.
Тепер розглянемо гамільтоніан $H_C$ для задачі **Max-Cut**. Нехай кожна вершина графа асоціюється з кубітом у стані $|0\rangle$ або $|1\rangle$, де значення позначає набір, у якому знаходиться вершина. Мета задачі – максимізувати кількість ребер $(v_1, v_2)$, для яких $v_1 = |0\rangle$ і $v_2 = |1\rangle$, або навпаки. Якщо ми асоціюємо оператор $Z$ з кожним кубітом, де

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

то ребро $(v_1, v_2)$ належить розрізу, якщо власне значення $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$; іншими словами, кубіти, асоційовані з $v_1$ і $v_2$, різні. Подібно, $(v_1, v_2)$ не належить розрізу, якщо власне значення $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$. Зауважте, що нас не цікавить точний стан кубіта, асоційований з кожною вершиною, натомість нас цікавить лише те, чи вони однакові чи ні на ребрі. Задача Max-Cut вимагає від нас знайти призначення кубітів на вершинах таким чином, щоб власне значення наступного гамільтоніана було мінімізовано
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Іншими словами, $b_i = 0$ для всіх $i$ у задачі Max-Cut. Значення $Q_{ij}$ позначає вагу ребра. У цьому посібнику ми розглядаємо невагомий граф, тобто $Q_{ij} = 1.0$ для всіх $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Крок 2: Оптимізація задачі для виконання на квантовому обладнанні
Контур вище містить серію абстракцій, корисних для роздумів про квантові алгоритми, але не можливих для запуску на обладнанні. Щоб мати змогу виконати на QPU, контур повинен пройти серію операцій, що складають крок **транспіляції** або **оптимізації контуру** патерну.

Бібліотека Qiskit пропонує серію **проходів транспіляції**, які задовольняють широкий спектр перетворень контуру. Вам потрібно переконатися, що Ваш контур **оптимізований** для Вашої мети.

Транспіляція може включати кілька кроків, таких як:

* **Початкове відображення** кубітів у контурі (таких як змінні рішення) на фізичні кубіти на пристрої.
* **Розгортання** інструкцій у квантовому контурі до нативних для обладнання інструкцій, які бекенд розуміє.
* **Маршрутизація** будь-яких кубітів у контурі, що взаємодіють, на фізичні кубіти, які є сусідніми один з одним.
* **Придушення помилок** шляхом додавання одно-кубітних воріт для придушення шуму з динамічною розв'язкою.

Більше інформації про транспіляцію доступно в нашій [документації](/guides/transpile).

Наступний код трансформує та оптимізує абстрактний контур у формат, готовий до виконання на одному з пристроїв, доступних через хмару, використовуючи **сервіс Qiskit IBM Runtime**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Крок 3: Виконання з використанням примітивів Qiskit
У робочому процесі QAOA оптимальні параметри QAOA знаходяться в ітеративному циклі оптимізації, який виконує серію оцінок контуру і використовує класичний оптимізатор для пошуку оптимальних параметрів $\beta_k$ і $\gamma_k$. Цей цикл виконання виконується через наступні кроки:

1. Визначити початкові параметри
2. Створити нову `Session`, що містить цикл оптимізації та примітив, що використовується для отримання вибірки з контуру
3. Після знаходження оптимального набору параметрів виконати контур фінальний раз для отримання фінального розподілу, який буде використовуватися на кроці постобробки.
#### Визначення контуру з початковими параметрами
Ми починаємо з довільно обраних параметрів.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Визначення бекенду та примітиву виконання
Використовуйте **примітиви Qiskit Runtime** для взаємодії з бекендами IBM&reg;. Двома примітивами є Sampler і Estimator, і вибір примітиву залежить від того, який тип вимірювання Ви хочете виконати на квантовому комп'ютері. Для мінімізації $H_C$ використовуйте Estimator, оскільки вимірювання функції вартості – це просто очікуване значення $\langle H_C \rangle$.
#### Запуск
Примітиви пропонують різноманітні [режими виконання](/guides/execution-modes) для планування робочих навантажень на квантових пристроях, і робочий процес QAOA виконується ітеративно в сесії.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Ви можете підключити функцію вартості на основі семплера до процедури мінімізації SciPy для пошуку оптимальних параметрів.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Після того, як Ви знайшли оптимальні параметри для контуру, Ви можете призначити ці параметри і отримати вибірку фінального розподілу, отриманого з оптимізованими параметрами. Тут слід використовувати примітив *Sampler*, оскільки це розподіл ймовірностей вимірювань бітових рядків, які відповідають оптимальному розрізу графа.

**Примітка:** Це означає підготовку квантового стану $\psi$ на комп'ютері з подальшим його вимірюванням. Вимірювання колапсує стан у один обчислювальний базисний стан – наприклад, `010101110000...` – який відповідає розв'язку-кандидату $x$ до нашої початкової задачі оптимізації ($\max f(x)$ або $\min f(x)$ залежно від завдання).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Крок 4: Постобробка та повернення результату в бажаному класичному форматі
Крок постобробки інтерпретує вихід вибірки для повернення розв'язку для Вашої оригінальної задачі. У цьому випадку Вас цікавить бітовий рядок з найвищою ймовірністю, оскільки це визначає оптимальний розріз. Симетрії в задачі дозволяють чотири можливі розв'язки, і процес вибірки поверне один з них з трохи вищою ймовірністю, але Ви можете побачити на графіку розподілу нижче, що чотири з бітових рядків є виразно більш вірогідними, ніж інші.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Візуалізація найкращого розрізу
З оптимального бітового рядка Ви можете потім візуалізувати цей розріз на оригінальному графі.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

І обчислити значення розрізу:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Частина II. Масштабування!
Ви маєте доступ до багатьох пристроїв з понад 100 кубітами на IBM Quantum&reg; Platform. Виберіть один з них для розв'язання задачі Max-Cut на зваженому графі зі 100 вузлами. Це задача "комунального масштабу". Кроки для побудови робочого процесу виконуються так само, як вище, але з набагато більшим графом.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Крок 1: Відобразити класичні вхідні дані на квантову задачу
#### Граф &rarr; Гамільтоніан
Спершу перетворіть граф, який Ви хочете розв'язати, безпосередньо в гамільтоніан, що підходить для QAOA.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Гамільтоніан &rarr; квантова схема

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Крок 2: Оптимізувати задачу для квантового виконання
Щоб масштабувати крок оптимізації схеми до задач комунального масштабу, Ви можете скористатися високопродуктивними стратегіями транспіляції, представленими в Qiskit SDK v1.0. Інші інструменти включають нову службу транспілятора з [проходами транспілятора, покращеними штучним інтелектом](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Коли оптимальні параметри від запуску QAOA на пристрої були знайдені, призначте параметри схемі.